# Vault Setup
This notebook has can start one of three versions of Vault. A Dockerized of Open Source Vault, a Dockerized Vault Enterprise version or a local version of Vault.

## Container
This sets up Vault (Enterprise) in a Docker container. Run either this or the localhost version below. 

In [27]:
#Set Env
export VAULT_PORT=8200
export VAULT_TOKEN=root
export VAULT_ADDR=http://localhost:${VAULT_PORT}
export VAULT_LICENSE=$(cat ../../license/vault.hclic)
# Set dir env vars. Create required directory.
MAIN_DIR=$(pwd)
WORK_DIR=config/vault
mkdir -p config/vault/{data,logs,config}

: 1

In [ ]:
# Clean up any running containers
docker stop vault; docker rm vault

In [ ]:
# docker pull hashicorp/vault-enterprise:latest
# docker pull hashicorp/vault:latest

### Start Vault Container

There are three methods below. Only run one of the methods.

* Vault OSS
* Vault Enterprise with default config
* Vault Enterprise with local config - this method will require initialization

In [ ]:
## Vault Opensource
docker run -d --rm --name vault \
    --cap-add=IPC_LOCK \
    -e "VAULT_DEV_ROOT_TOKEN_ID=${VAULT_TOKEN}" \
    -e "VAULT_DEV_LISTEN_ADDRESS=:${VAULT_PORT}" \
    -e "VAULT_ADDR=$VAULT_ADDR" \
    -p ${VAULT_PORT}:${VAULT_PORT} \
    hashicorp/vault:latest

In [ ]:
## Vault Enterprise
docker run -d --rm --name vault \
    --cap-add=IPC_LOCK \
    -e "VAULT_DEV_ROOT_TOKEN_ID=${VAULT_TOKEN}" \
    -e "VAULT_DEV_LISTEN_ADDRESS=:${VAULT_PORT}" \
    -e "VAULT_ADDR=$VAULT_ADDR" \
    -p ${VAULT_PORT}:${VAULT_PORT} \
    hashicorp/vault-enterprise:latest

In [49]:
cat << EOF > docker-compose.yaml
version: '3.7'

services:
  consul-server1:
    image: hashicorp/consul:1.10.0
    container_name: consul-server1
    restart: always
    volumes:
     - ./server1.json:/consul/config/server1.json:ro
     - ./certs/:/consul/config/certs/:ro
    networks:
      - hashi
    ports:
      - "8500:8500"
      - "8600:8600/tcp"
      - "8600:8600/udp"
#     command: "agent -bootstrap-expect=3"

#   consul-client:
#     image: hashicorp/consul:1.10.0
#     container_name: consul-client
#     restart: always
#     volumes:
#      - ./client.json:/consul/config/client.json:ro
#      - ./certs/:/consul/config/certs/:ro
# #    networks:
# #      - hashi
#     command: "agent"

  vault:
    image: hashicorp/vault:1.7.3 # hashicorp/vault-enterprise:latest
    container_name: vault
    restart: always
    volumes:
      - ./config/vault/data:/vault/data
      - ./config/vault/logs:/etc/vault/logs
      - ./config/vault/config:/vault/config:ro
    networks:
      - hashi
    ports:
      - "8200:8200/tcp"
    environment:
      VAULT_DEV_ROOT_TOKEN_ID: 'root2'
#       VAULT_DEV_LISTEN_ADDRESS: '0.0.0.0:8200'
      VAULT_ADDR: ${VAULT_ADDR}
#       VAULT_LOCAL_CONFIG: >-
#         {
#           "backend": {
#             "consul": {
#               "address": "consul-dr:9500",
#               "path": "vaultdr/"
#             }
#           },
#           "listener": {
#               "tcp": {
#                   "address": "0.0.0.0:8200",
#                   "tls_cert_file": "/vault/config/server.crt",
#                   "tls_key_file": "/vault/config/server.pem",
#                   "tls_disable_client_certs": "true"
#               }
#           },
#           "default_lease_ttl": "168h",
#           "max_lease_ttl": "720h",
#           "ui": true
#         }
    cap_add:
      - IPC_LOCK
    entrypoint: "vault server -dev -config=/vault/config"

networks:
 hashi:
   driver: bridge
EOF

: 1

In [45]:
cat << EOF > $MAIN_DIR/$WORK_DIR/config/vault1.hcl
storage "file" {
    path = "/vault/file"
}

# listener "tcp" {
#   address       = "0.0.0.0:8200"
# #   tls_cert_file = "/path/to/fullchain.pem"
# #   tls_key_file  = "/path/to/privkey.pem"
# }

default_lease_ttl = "168h"
max_lease_ttl = "0h"
api_addr = "http://0.0.0.0:8200"

# seal "pkcs11" {
#   lib            = "/usr/vault/lib/libCryptoki2_64.so"
#   slot           = "0"
#   pin            = "AAAA-BBBB-CCCC-DDDD"
#   key_label      = "vault-hsm-key"
#   hmac_key_label = "vault-hsm-hmac-key"
# }

telemetry {
  prometheus_retention_time = "30s"
  disable_hostname = true
}
EOF

: 1

In [50]:
# docker compose down
docker compose up -d vault

[+] Building 0.0s (0/0)                                                         
[+] Building 0.0s (0/1)                                                         
[+] Building 0.2s (2/3)                                                         
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 63B                                        0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for docker.io/hashicorp/vault:1.7.3           0.1s
[+] Building 0.3s (2/3)                                                         
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 63B                                        0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B     

: 1

In [ ]:
## Vault Enterprise - local config
docker run -d --rm --name vault \
    --cap-add=IPC_LOCK \
    -e "VAULT_DEV_ROOT_TOKEN_ID=${VAULT_TOKEN}" \
    -e "VAULT_DEV_LISTEN_ADDRESS=0.0.0.0:${VAULT_PORT}" \
    -e "VAULT_ADDR=$VAULT_ADDR" \
    -p ${VAULT_PORT}:${VAULT_PORT} \
    -e "VAULT_LOCAL_CONFIG=$(cat config.hcl)" \
    hashicorp/vault-enterprise:latest

## localhost
This sets up Vault (Enterprise) on the local host. Run either this or the Docker version above. 

In [ ]:
#Set Env
export VAULT_PORT=8201
export VAULT_TOKEN=root
export VAULT_ADDR=http://localhost:${VAULT_PORT}
export VAULT_LICENSE=$(cat ../../license/vault.hclic)

In [ ]:
vault server \
  -dev -dev-root-token-id=${VAULT_TOKEN} -dev-listen-address=:8201 \
  -log-level=trace \
  -config=./config.hcl   > vault.log 2>&1 &

## Status, Audit, and License

In [ ]:
vault status

In [ ]:
# Enable audit output to file, audit settings are in config.hcl
vault audit enable file file_path=/vault/logs/audit.log

In [ ]:
## If setting up Vault Enterprise, place license key file in ../../license/vault.hclic
# sleep 5
curl \
    --silent \
    --header "X-Vault-Token: $VAULT_TOKEN" \
    --request PUT \
    --data '{"text":"'$VAULT_LICENSE'"}' \
    $VAULT_ADDR/v1/sys/license

## UI

In [ ]:
open $VAULT_ADDR
echo "login with token: $VAULT_TOKEN"

## Sample Authentication Methods

* For LDAP configuration, see [LDAP](./110-Setup-authmethods.ipynb#LDAP)
* For User Pass, see [UserPass](./110-Setup-authmethods.ipynb#UserPass)

# Clean Up

In [ ]:
docker stop vault-enterprise vault
docker rm vault-enterprise vault
docker ps -a
pkill vault